In [3]:
#my imports, to make sure it works fine on Jupyter Notebook

import sys
import warnings
import re
import json
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import SparkSession
# from pyspark.sql.context import SQLContext
# from pyspark.sql.functions import udf
# from pyspark.sql.functions import lit
# from pyspark.sql.window import Window
# import pyspark.sql.functions as F
# from datetime import datetime
# import pyspark.sql.types as T 
# from pyspark.sql.functions import split, explode

#To get spark. working without throwing a NameError
import findspark
findspark.init()
import pyspark as ps # Call this only after findspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
# sc = SparkContext.getOrCreate()
# spark = SparkSession(sc)


# In[ ]:


#ACTUAL FUNCTION

#for renaming the columns
from functools import reduce

#allow us to use SQL Count function for aggregation in groupBY
from pyspark.sql.functions import count

#allow us to read csv to dataframe
import pandas as pd #need Pandas
from pyspark.sql.types import *

In [36]:
mapping = testing = spark.createDataFrame(
    [
        ("Pop Goes the Weasel2", "2:00"), 
    ],
    ['Title', 'Length'] # add your columns label here
)



In [72]:
from datetime import datetime, timedelta

t = datetime.strptime("2:00", "%M:%S")
delta = timedelta(minutes=t.minute, seconds = t.second)

testing = spark.createDataFrame(
    [
        (1, "Pop Goes the Weasel2.mp3", 
         datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p'), 
         datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p'),
        False), # create your data here, be consistent in the types.
        (1, "Pop Goes the Weasel2.mp3", 
         datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p'), 
         datetime.strptime('Jun 1 2005  1:34PM', '%b %d %Y %I:%M%p'),
        True),
    ],
    ['device_id', 'item_name', "start", "end", "reach_end_of_stream"] # add your columns label here
)

testing.show()

+---------+--------------------+-------------------+-------------------+-------------------+
|device_id|           item_name|              start|                end|reach_end_of_stream|
+---------+--------------------+-------------------+-------------------+-------------------+
|        1|Pop Goes the Weas...|2005-06-01 13:33:00|2005-06-01 13:37:00|              false|
|        1|Pop Goes the Weas...|2005-06-01 13:33:00|2005-06-01 13:34:00|               true|
+---------+--------------------+-------------------+-------------------+-------------------+



In [108]:
testing.write.parquet('input.parquet')

In [110]:
blank_df = spark.range(0).drop("id")

++
||
++
++



In [111]:
blank_df.write.parquet("output.parquet")

In [113]:
mapping.write.parquet("contentMapping.parquet")

In [106]:
import pyspark.sql.functions as F
from pyspark.sql.functions import udf 
from pyspark.sql.types import IntegerType


def main(inputFile, outputFile, configFile, contentMapping):
    df = testing
    contentMapping = mapping
    
    #get rid of ".mp3" in item_name
    df = df.withColumnRenamed("item_name", "to_del")
    df = df.withColumn("item_name", F.split(df['to_del'], '\.')[0])
    df = df.drop('to_del')
    
    #turn Content Mapping String Length to TimeDelta Object
    strp_time = udf (lambda x: datetime.strptime(x, "%M:%S"))
    time_delta = udf (lambda y: timedelta(minutes = y.minute, seconds = y.second))
    
    contentMapping = contentMapping.withColumn("strptime", strp_time(F.col("Length")))
    contentMapping = contentMapping.withColumn("Content Length", time_delta(F.col("strptime")))
    contentMapping = contentMapping.drop('strpTime')
    contentMapping = contentMapping.withColumnRenamed("Title", "item_name")
    
    #Merge df and contentMapping
    df = df.join(contentMapping, ["item_name"], "outer")
    
    #get time played for
    df = df.withColumn("Played For", F.unix_timestamp(df["end"]) - F.unix_timestamp(df["start"]))
    
    #get total seconds of song as String, convert to bigInt
    df = df.withColumn("Song Duration Str", F.regexp_extract(df["Content Length"], "(?<=total: )(.*)(?= seconds)", 0))
    df = df.withColumn("Song Duration Int", df["Song Duration Str"].cast(IntegerType()))
    

    #Let's get Percentage Played
    df = df.withColumn("Percent Played", df["Played For"] / df["Song Duration Int"])
    
    #Let's keep only the columns we need at this point
    df = df.select(["device_id", "item_name", "Percent Played"])

    #assign weights based on Percent Played
    df = df.withColumn(
    'weight',
    F.when((F.col("Percent Played") >= 0.0) & (F.col("Percent Played") < 0.25), 0.0)\
    .when((F.col("Percent Played") >= 0.25) & (F.col("Percent Played") < 0.50), 0.33)\
    .when((F.col("Percent Played") >= 0.50) & (F.col("Percent Played") < 0.75), 0.66)\
    .otherwise(1.0)
    )

    
    return df.show(2, False)
    
    #df = spark.read.parquet(inputFile+'/*').dropDuplicates().na.drop() Read from inputFile
    #df.write.parquet(outputFile) Write onto output Parquet
main("dd", "dd", "dd", "dd")

+---------+--------------------+--------------+------+
|device_id|item_name           |Percent Played|weight|
+---------+--------------------+--------------+------+
|1        |Pop Goes the Weasel2|2.0           |1.0   |
|1        |Pop Goes the Weasel2|0.5           |0.66  |
+---------+--------------------+--------------+------+



In [ ]:
### common pyspark import statements
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.context import SQLContext
from pyspark.sql.functions import udf 
from pyspark.sql.functions import lit
from pyspark.sql.window import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T 
from pyspark.sql.functions import split, explode
### other essnetial import statements
import argparse
from configparser import ConfigParser
from datetime import datetime
from datetime import timedelta

#my imports
from pyspark.sql.types import IntegerType

'''
	Standardize code to the following structure: everything to be in functions except pyspark envrionment setup
		- environment setup
		- user defined functions
		- main function
		- program start point
''' 

# setup
conf = SparkConf()
conf.setAppName('pillar')
sc = SparkContext(conf=conf)
sc.setLogLevel('WARN')
sql_context = SQLContext(sc)
spark = SparkSession.builder.appName('pillar').getOrCreate()

# user defined functions

def userDefinedFunction1():
    return

def userDefinedFunction2():
    return


# main function: should always take in input file, output file and config file
def main(inputFile, outputFile, configFile, contentMapping):
    df = spark.read.parquet(inputFile+'/*').dropDuplicates().na.drop()
    contentMapping = spark.read.parquet(contentMapping+'/*') #right formatting?
    
    #get rid of ".mp3" in item_name
    df = df.withColumnRenamed("item_name", "to_del")
    df = df.withColumn("item_name", F.split(df['to_del'], '\.')[0])
    df = df.drop('to_del')
    
    #turn Content Mapping String Length to TimeDelta Object
    strp_time = udf (lambda x: datetime.strptime(x, "%M:%S"))
    time_delta = udf (lambda y: timedelta(minutes = y.minute, seconds = y.second))
    
    contentMapping = contentMapping.withColumn("strptime", strp_time(F.col("Length")))
    contentMapping = contentMapping.withColumn("Content Length", time_delta(F.col("strptime")))
    contentMapping = contentMapping.drop('strpTime')
    contentMapping = contentMapping.withColumnRenamed("Title", "item_name")
    
    #Merge df and contentMapping
    df = df.join(contentMapping, ["item_name"], "outer")
    
    #get time played for
    df = df.withColumn("Played For", F.unix_timestamp(df["end"]) - F.unix_timestamp(df["start"]))
    
    #get total seconds of song as String, convert to bigInt
    df = df.withColumn("Song Duration Str", F.regexp_extract(df["Content Length"], "(?<=total: )(.*)(?= seconds)", 0))
    df = df.withColumn("Song Duration Int", df["Song Duration Str"].cast(IntegerType()))
    

    #Let's get Percentage Played
    df = df.withColumn("Percent Played", df["Played For"] / df["Song Duration Int"])
    
    #Let's keep only the columns we need at this point
    df = df.select(["device_id", "item_name", "Percent Played"])

    #assign weights based on Percent Played
    df = df.withColumn(
    'weight',
    F.when((F.col("Percent Played") >= 0.0) & (F.col("Percent Played") < 0.25), 0.0)\
    .when((F.col("Percent Played") >= 0.25) & (F.col("Percent Played") < 0.50), 0.33)\
    .when((F.col("Percent Played") >= 0.50) & (F.col("Percent Played") < 0.75), 0.66)\
    .otherwise(1.0)
    )

    df.write.parquet(outputFile) # Write onto output Parquet


# program start point
if __name__ == "__main__":
    ''' 
        Add arguments to script during execution on command line
        Example of how to run the script: spark-submit template.py -i input.parquet -o output.parquet -c config.ini
    '''
    parser = argparse.ArgumentParser() 
    parser.add_argument('-i', '--input', required=True)
    parser.add_argument('-o', '--output', required=True)
    parser.add_argument('-c', '--config', required=True)
    parser.add_argument('-cm', '--contentmapping', required=True) #added on, so contentMapping is required
    args = parser.parse_args()
    config = ConfigParser()
    config.read(args.config)
    main(args.input,args.output,config, args.contentmapping)